In [254]:
import numpy as np
import copy
def get_number_of_vertices(mesh):
    string_to_match = 'Vertices'
    for index, line in enumerate(mesh):
        line = line.rstrip()  # Need to remove the \n at the end of the line
        if line == string_to_match:
            print("Number of vertices: ", mesh[index+1])
            break
    
    return [index,int(mesh[index+1])]

def get_number_of_triangles(mesh):
    string_to_match = 'Triangles'
    for index, line in enumerate(mesh):
        line = line.rstrip() 
        if line == string_to_match:
            print("Number of triangles: ", mesh[index+1])
            break
    
    return [index,int(mesh[index+1])]

    
def read_text(file):
    with open(file,'r') as f:
        for line in f:
            print(line)

    
mymesh = open('mesh.mesh','r')
mesh_text = mymesh.readlines()


[iv,v] = get_number_of_vertices(mesh_text)
[it,t] = get_number_of_triangles(mesh_text)
print(v,iv,'vertices and',t,it, 'triangles')

list_of_vertices = np.empty(shape = [0,4])
list_of_triangles = np.empty(shape = [0,4])
new_vertices = np.empty(shape = [0,4])
new_triangles = np.empty(shape = [0,4])



# Obtaining the list of vertices:

for i in range(v):
    x = np.array(mesh_text[iv+i+2].rstrip().split())
    y = x.astype(float)
    list_of_vertices = np.vstack((list_of_vertices, y))

# Obtaining the list of triangles
for i in range(t):
    x_t = np.array(mesh_text[it+i+2].rstrip().split())
    y_t = x_t.astype(int)
    list_of_triangles = np.vstack((list_of_triangles, y_t))


# Symmetry operation
# Symmetry around the yz-plane means that each point x = (x1, x2, x3), will be mapped onto (-x1,x2,x3)
# Symmetry around the xz-plane means that each point x = (x1, x2, x3), will be mapped onto (x1,-x2,x3)
# Symmetry around the xy-plane means that each point x = (x1, x2, x3), will be mapped onto (x1,x2,-x3)

# Symmetry around xy-plane

new_vertices = copy.deepcopy(list_of_vertices)
new_vertices[:,0]*=-1

new_triangles = copy.deepcopy(list_of_triangles)
new_triangles[:,0]+=v
new_triangles[:,1]+=v
new_triangles[:,2]+=v
# print(new_triangles)
# print(list_of_triangles)


Number of vertices:  771

Number of triangles:  1431

771 3 vertices and 1431 777 triangles


In [253]:
out_file = 'out.mesh'


f = open(out_file, 'w')
f.write(mesh_text[0])
f.write(mesh_text[1])
f.write(mesh_text[2])
f.write('Vertices\n')
f.write(str(2*v)+'\n')
for line in list_of_vertices:
    y=line.astype(str)
    f.write(y[0]+ ' '+ y[1] + ' ' + y[2]+ ' ' + y[3]+ '\n')
for line in new_vertices:
    y=line.astype(str)
    f.write(y[0]+ ' '+ y[1] + ' ' + y[2]+ ' ' + y[3]+ '\n')

f.write('\n')
f.write('Triangles'+ '\n')
f.write(str(2*t)+ '\n')

for line in list_of_triangles:
    y = [int(i) for i in line]
    f.write(str(y[0])+ ' '+ str(y[1]) + ' ' + str(y[2])+ ' ' + str(y[3])+ '\n')

for line in new_triangles:
    y = [int(i) for i in line]
    f.write(str(y[0])+ ' '+ str(y[1]) + ' ' + str(y[2])+ ' ' + str(y[3])+ '\n')

f.write("End"+'\n')
f.close()

# Reading what I just created

mymesh = open('test.mesh','r')
# read_text('out.mesh')
